In [0]:
import zipfile
import os
from pyspark.sql import SparkSession

# Caminhos
zip_path = "/Volumes/nycproject/default/nyc_volume/nyc_pipeline.zip"
extract_path = "/Volumes/nycproject/default/nyc_volume/nyc_pipeline"

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("Arquivos extraídos:")
print(os.listdir(extract_path))

# Spark session
spark = SparkSession.builder.appName("Ingestao Bronze").getOrCreate()

input_path = "dbfs:/Volumes/nycproject/default/nyc_volume/nyc_pipeline/*.csv"
bronze_path = "dbfs:/Volumes/nycproject/default/bronze/nyc_taxi"

df = spark.read.option("header", "true").csv(input_path)
df.write.format("delta").mode("overwrite").save(bronze_path)

spark.sql("USE CATALOG nycproject")
spark.sql("USE SCHEMA default")
spark.sql("""
CREATE TABLE IF NOT EXISTS nyc_taxi_bronze
USING DELTA
AS SELECT * FROM delta.`/Volumes/nycproject/default/bronze/nyc_taxi`
""")




In [0]:
%fs ls /Volumes/nycproject/default/nyc_volume/nyc_pipeline/

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Verifica Extração").getOrCreate()

# Caminho no DBFS
csv_path = "dbfs:/Volumes/nycproject/default/nyc_volume/nyc_pipeline/*.csv"

# Leitura
df = spark.read.option("header", "true").csv(csv_path)


In [0]:
# Exibir esquema 
display(df)